In [1]:
import collections
import copy

def ld():
    return [int(x) for x in open("23.txt").read().split(',')]

class m:
    idgen=0
    
    #get op and modes of following 3 addresses
    def ix(s,x):
        assert(x>=0)
        x="%05d"%(x)
        op=int(x[3:])
        m3,m2,m1=[int(c) for c in x[:3]]
        return op,m1,m2,m3


    def gv(s,m,ix):
        assert(ix in [1,2,3])
        if m==0:#reg
            return s.p[s.p[s.ip+ix]]
        elif m==1:#imm
            return s.p[s.ip+ix] 
        elif m==2:#reg+rb
            return s.p[s.p[s.ip+ix]+s.rb]
        assert(0)
        
    def __init__(s,p):
        #load to mem
        s.p=collections.defaultdict(int)
        for i,v in enumerate(p):
            s.p[i]=v
            
        
        #input queue
        s.i=[]
        
        #id
        s.id=m.idgen
        m.idgen+=1
        
        s.ip=0
        
        #print("M",s.id,"created")
        
        #relative base
        s.rb=0
        
        #count all vm ticks
        s.ticks=0
        
        s.halt=False
        
        s.st=0
        s.tgt=0
        
        s.idle=0
    
    def sendInput(s, i):
        s.i.append(i)
        
     
    def tick(s):
        r=None
        
        if s.halt:
            print("halted")
            return r
        op,m1,m2,m3=s.ix(s.p[s.ip])
        if op==1:#add
            assert(m3 in [0,2])
            tix=s.p[s.ip+3]
            if m3==2:
                tix+=s.rb
            s.p[tix]=s.gv(m1,1)+s.gv(m2,2)
            s.ip+=4
        elif op==2:#mul
            assert(m3 in [0,2])
            tix=s.p[s.ip+3]
            if m3==2:
                tix+=s.rb
            s.p[tix]=s.gv(m1,1)*s.gv(m2,2)
            s.ip+=4
        elif op==3:#in
            assert(m1 in [0,2])
            iv=None
            if len(s.i)>0:
                iv=s.i[0]
                s.i=s.i[1:]
            else:
                iv=-1
                s.idle+=1
            tix=s.p[s.ip+1]
            if m1==2:
                tix+=s.rb
            s.p[tix]=iv
            s.ip+=2
        elif op==4:#out
            r=s.gv(m1,1)
            #print(s.id,"spits out",r)
            s.ip+=2
            s.idle=0
            #print("--id:",s.id,"onwards")
        elif op==5:#jump-if-true
            if s.gv(m1,1):
                s.ip=s.gv(m2,2)
            else:
                s.ip+=3
        elif op==6:#jump-if-false
            if not s.gv(m1,1):
                s.ip=s.gv(m2,2)
            else:
                s.ip+=3
        elif op==7:#less-than
            assert(m3 in [0,2])
            tix=s.p[s.ip+3]
            if m3==2:
                tix+=s.rb
            s.p[tix]=s.gv(m1,1)<s.gv(m2,2)
            s.ip+=4
        elif op==8:#equals
            assert(m3 in [0,2])
            tix=s.p[s.ip+3]
            if m3==2:
                tix+=s.rb
            s.p[tix]=s.gv(m1,1)==s.gv(m2,2)
            s.ip+=4
        elif op==9:#adjust relative base
            s.rb+=s.gv(m1,1)
            s.ip+=2
        elif op==99:#end
            s.halt = True
            print("\nfin", s.id)
            r=0
            
        else:
            #pass
            #print("id", s.id, "op",op,"c",c)
            assert(0)
        #print(op)
        s.ticks+=1
        return r
    

    
    

In [2]:
import copy
import time

def p1():
    N=50
    net = {}
    
    for i in range(50):
        net[i] = {}
        net[i]["machine"] = m(ld())
        net[i]["machine"].sendInput(i)
        net[i]["output"] = []
    
    ticks=0
    nextprint=0
    while ticks < 10e6:
        if ticks > nextprint:
            nextprint = ticks + 1000
            print("ticks: %d"%(ticks),end='\r')
        n=1
        for i in net.keys():
            r=net[i]["machine"].tick()
            
            if not r is None:
                #print(r)
                net[i]["output"].append(r)
                if len(net[i]["output"])==3:
                    a,x,y=net[i]["output"]
                    #print("%d->%d : %d,%d."%(i,a,x,y))
                    if a==255:
                        return y
                    #net[a]["machine"].sendInput(a)
                    net[a]["machine"].sendInput(x)
                    net[a]["machine"].sendInput(y)
                    net[i]["output"]=[]
        ticks+=1
        
    
   
print("\npart1:", p1(),22134)




ticks: 1002
part1: 22134 22134


In [3]:

def p2():
    N=50
    net = {}
    X=0
    Y=0
    lastY=-1
    frsh=0
    for i in range(50):
        net[i] = {}
        net[i]["m"] = m(ld())
        net[i]["m"].sendInput(i)
        net[i]["o"]=[]
    ticks=0
    nextprint=0
    while ticks < 10e6:
        #if ticks > nextprint:
        #    nextprint = ticks + 1000
        #    print("ticks: %d"%(ticks),end='\r')
        
        
        for i in net.keys():
            r=net[i]["m"].tick()
            
            if not r is None:
                net[i]["o"].append(r)
                if len(net[i]["o"])==3:
                    a,x,y=net[i]["o"]
                    if a==255:
                        print("pickup",x,y,end='\r')
                        X=x
                        Y=y
                        frsh=1
                        #print(x,y)
                    else:
                        net[a]["m"].sendInput(x)
                        net[a]["m"].sendInput(y)
                    net[i]["o"]=[]
                
        idle=True        
        for i in net.keys():        
            if net[i]["m"].idle < 200:
                idle=0
                    
                
                
        if idle:
            #print("\nidle")
            net[0]["m"].sendInput(X)
            net[0]["m"].sendInput(Y)
            if Y==lastY and frsh:
                
                #print(Y)
                return Y
                pass
            frsh=0
            lastY=Y
        ticks+=1
        
    


print("\npart2:", p2(),16084)
#22134 too high
#20540 too high




pickup 30757 16084
part2: 16084 16084
